# Lobito Corridor Spatial Analysis
----

Benny Istanto, bistanto@worldbank.org, GOST/DEC Data Group

Objective. Produce fast, reproducible **access & opportunity** analytics using only aligned rasters: 
1. Isochrone coverage KPIs (population, cropland, electrification), 
2. A transparent **priority surface** for smallholder aggregation/feeder upgrades, 
3. A **flood-aware road risk** screening, and 
4. **Site cards** for the project locations.
              
**Expected outputs.** AOI-prefixed GeoTIFFs in `outputs/rasters/`, CSV tables in `outputs/tables/`, and quick PNG maps in `outputs/figs/`.

## Run once after (re)starting the kernel


<div class="alert alert-block alert-success" align="left">
  <b>Why this cell?</b><br/>
  It initializes the workspace for the current AOI—sets <code>PROJECT_ROOT</code>/<code>AOI</code>, adds <code>src/</code> to <code>sys.path</code>, clears the import cache, and loads <code>config.py</code> so <code>PATHS</code>/<code>PARAMS</code> are available to all steps.
  <br/><br/>
  <b>When to run it</b>
  <ul>
    <li>After a kernel restart (once).</li>
    <li>After changing <code>AOI_VALUE</code> or editing <code>config.py</code> (paths/parameters/templates).</li>
    <li>After moving the project folder.</li>
  </ul>
  <b>Editing step code?</b> Re-import the step and <code>importlib.reload(module)</code> before calling <code>main()</code>.
</div>


In [1]:
# Bootstrap imports for package-style repo
import os, sys, importlib
from pathlib import Path

# >>>> EDIT THESE TWO ONLY <<<<
PROJECT_ROOT = Path("/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis")
AOI_VALUE = "huambo"  # e.g., "benguela", "huambo", "bei", "moxico", "moxicoleste"

# Environment for the codebase
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
os.environ["AOI"] = AOI_VALUE

src_path = PROJECT_ROOT / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Make sure Python sees any file edits we made
importlib.invalidate_caches()

from config import PATHS, PARAMS, AOI, get_logger
print("AOI:", AOI)
PATHS.OUT, PATHS.OUT_R, PATHS.OUT_T, PATHS.OUT_F


AOI: huambo


(PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs'),
 PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/rasters'),
 PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables'),
 PosixPath('/mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/figs'))

## Step 00 — Data alignment & vector→raster

Align everything to a single 1-km template so every pixel stacks perfectly. This step converts key vectors (cropland, electrification, settlements) to rasters, harmonizes units/CRS, and prepares a 1-km flood screening layer alongside the native 30 m flood depth for engineering checks.

**Objective.** Produce AOI-aligned base rasters and vector-to-raster layers for consistent downstream math.

**Inputs.**
* `PARAMS.TARGET_GRID` (template), raw rasters (POP/NTL/VEG/drought), flood depth (30 m), and vectors (cropland, electrification, settlements).
  
**Outputs.**
* `{AOI}_pop_1km.tif`, `{AOI}_ntl_1km.tif`, `{AOI}_veg_1km.tif`, `{AOI}_drought_1km.tif`,
* `{AOI}_cropland_1km.tif`, `{AOI}_electric_1km.tif`, `{AOI}_settlement_1km.tif`,
* `{AOI}_flood_maxdepth_30m.tif` + `{AOI}_flood_maxdepth_1km.tif`,
* `{AOI}_rwi_meta_1km.tif` (if available in your data bundle).
  
**Assumptions & design.**
* Everything is reprojected/matched to the template; vector rasterization uses `all_touched=True`.
* Nodata is respected; logging warns on missing optional layers.

**Knobs.** 
* In `config.py`: source paths, `PARAMS.TARGET_GRID`.


In [2]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_00_align_and_rasterize"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


21:19:21 | INFO | Loaded target grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083 (units of CRS)
21:19:21 | INFO | Aligning rasters to target grid...
21:19:21 | INFO | Wrote huambo_pop_1km.tif
21:19:22 | INFO | Wrote huambo_ntl_1km.tif
21:19:22 | INFO | Wrote huambo_veg_1km.tif
21:19:22 | INFO | Wrote huambo_drought_1km.tif
21:19:22 | INFO | Aligning Meta Relative Wealth Index (RWI) to 1-km grid...
21:19:22 | INFO | Wrote huambo_rwi_meta_1km.tif
21:19:22 | INFO | Rasterizing cropland presence (1=any in cell)...
21:20:40 | INFO | Wrote huambo_cropland_presence_1km.tif
21:20:40 | INFO | Rasterizing cropland fraction (0..1 per 1-km cell, supersample=10)...
21:21:57 | INFO | Wrote huambo_cropland_fraction_1km.tif | mean_fraction=0.168
21:22:00 | INFO | Rasterizing electricity masks using field 'FinalElecC' (1=grid, 99=unelectrified)...
21:22:04 | INFO | Wrote huambo_elec_grid_1km.tif, huambo_elec_unelectrified_1km.tif
21:22:06 | INFO | Rasterizing settlement type using field 'IsUrba

## Step 01 — Isochrones (accumulated travel time)

Turn road/friction inputs into a travel-time surface in minutes. This becomes the backbone for coverage masks (≤30/60/120 min) and access KPIs.

**Objective.** Compute the accumulated cost (minutes) raster.
                                                                                                               
**Inputs.**
* `PARAMS.TARGET_GRID`, friction/network inputs per `config.py`.
  
**Outputs.**
* `{AOI}_traveltime_min.tif` (accumulated cost surface).
  
**Assumptions.**
* Minutes as the canonical unit; obstacles/penalties embedded in friction.
  
**Knobs.**
* Friction parameters in `PARAMS` (if exposed), thresholds in `PARAMS.ISO_THRESH` (e.g., `(30, 60, 120)`).


In [3]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_01_isochrones"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


21:23:29 | INFO | Isochrones from ago_phy_huambo_traveltime_market.tif | CRS=EPSG:4326 | size=280x216 | cell=0.0x0.0
21:23:29 | INFO | Building ≤30 min isochrone...
21:23:29 | INFO | Wrote huambo_iso_le_30min_1km.tif | cells inside: 2230/39751 (5.6%)
21:23:29 | INFO | Building ≤60 min isochrone...
21:23:29 | INFO | Wrote huambo_iso_le_60min_1km.tif | cells inside: 7215/39751 (18.2%)
21:23:29 | INFO | Building ≤120 min isochrone...
21:23:29 | INFO | Wrote huambo_iso_le_120min_1km.tif | cells inside: 21964/39751 (55.3%)
21:23:29 | INFO | Building ≤240 min isochrone...
21:23:29 | INFO | Wrote huambo_iso_le_240min_1km.tif | cells inside: 36738/39751 (92.4%)
21:23:29 | INFO | Step 01 complete.


## Step 02 — KPIs for isochrone coverage (population, cropland, electrification)

Quantify how much population, cropland, and electrification lie within each time threshold. These coverage KPIs are the simplest way to communicate “who’s connected.”

**Objective.** Summarize POP/CROP/ELECTRIC inside `tt ≤ threshold`.

**Inputs.**
* `{AOI}_traveltime_min.tif`, 1-km POP/CROPLAND/ELECTRIC rasters.
  
**Outputs.**
* `tables/{AOI}_iso_kpis.csv` (per threshold).
  
**Assumptions.**
* Coverage mask: `tt_minutes <= threshold`.
  
**Knobs.**
* `PARAMS.ISO_THRESH` list.


In [4]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_02_kpis_population_cropland_electric"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


21:23:33 | INFO | Electrification grid value counts (non-NaN): {np.int64(1): 588}
21:23:33 | INFO | KPI base rasters loaded | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
21:23:33 | INFO | Electrification mapping: FinalElecCode2020 -> elec_grid_1km == 1 (grid); 99/0/NaN = non-grid.
21:23:33 | INFO | Denominators | pop_total=2,997,196 | cropland_total_km2=10,156.91 | electrified_cells_total=588 | cell_area_km2≈1.000
21:23:33 | INFO | ≤ 30 min | pop=1,301,172 (43.4%) | crop=879.63 km² (8.7%) | elec_cells=414 (70.4%)
21:23:33 | INFO | ≤ 60 min | pop=1,635,860 (54.6%) | crop=2,212.28 km² (21.8%) | elec_cells=448 (76.2%)
21:23:33 | INFO | ≤120 min | pop=2,438,910 (81.4%) | crop=5,996.37 km² (59.0%) | elec_cells=563 (95.7%)
21:23:33 | INFO | ≤240 min | pop=2,920,311 (97.4%) | crop=9,485.93 km² (93.4%) | elec_cells=584 (99.3%)
21:23:33 | INFO | Saved KPI table → /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables/huambo_kpis_isochrones.csv
21:23:33 | I

## Step 03 — Priority surface (baseline)

Combine normalized indicators: better access (lower minutes), higher population & vegetation, stronger nighttime lights, and lower drought frequency. Save the score raster and a top-decile mask to highlight quick-win areas.

**Objective.** (Baseline) Build a transparent, minimal priority surface from core overlays.

**Inputs.**
* 1-km standardized layers (POP/CROP/ELECTRIC etc.).
  
**Outputs.**
* `rasters/{AOI}_priority_score_0_1.tif`, `rasters/{AOI}_priority_top10_mask.tif`.
  
**Assumptions.**
* Simpler normalization and weights vs Step 07.
  
**Recommendation.**
* Keep Step 03 as **baseline** only; prefer Step 07 for production.


In [5]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_03_priority_surface"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


21:23:43 | INFO | Priority surface inputs loaded | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
21:23:43 | INFO | Weights | ACC=0.35 POP=0.25 VEG=0.20 NTL=0.10 DRT=0.10
21:23:43 | INFO | Wrote huambo_priority_score_0_1.tif
21:23:43 | INFO | Wrote huambo_priority_top10_mask.tif | P90=0.759 | top10 cells: 283/2826 (10.0%)
21:23:43 | INFO | Step 03 complete.


## Step 04 — Flood bottlenecks from road raster

Flag where roads intersect flood depths beyond thresholds. Use the 30 m layer for engineering plausibility, and the 1 km “max depth” for corridor-scale screening.

**Objective.** Identify likely flood bottlenecks impacting road access.

**Inputs.**
* 30 m flood depth, road raster or vector.
  
**Outputs.**
* `rasters/{AOI}_flood_bottlenecks_1km.tif` (or CSV summary, depending on implementation).
  
**Assumptions.**
* Engineering check: depth thresholds configurable.
  
**Knobs.**
* Flood depth threshold(s) in `PARAMS` (if exposed).


In [7]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_04_flood_bottlenecks_from_road_raster"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
importlib.reload(m)

m.main()


21:57:41 | WARNING | Reprojected to match target grid: flood1k, prio10
21:57:41 | INFO | Flood screening inputs loaded | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
21:57:41 | INFO | Params | FLOOD_DEPTH_RISK=0.3 m | ROAD_CLASSES_KEEP=None
21:57:41 | INFO | Rasterizing roads: no class filter (use all OSM fclass values).
21:57:43 | INFO | Wrote huambo_roads_main_1km.tif | road_cells=13,194
21:57:43 | INFO | Wrote huambo_roads_flood_risk_cells_1km.tif | risk_cells=12,128
21:57:43 | INFO | Wrote huambo_roads_flood_risk_near_priority_1km.tif | risk_near_priority_cells=357
21:57:43 | INFO | Saved summary → /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables/huambo_roads_flood_risk_summary.csv
21:57:43 | INFO | Step 04 complete.


## Step 05 — Site audit points

Build “site cards” by sampling rasters around project points. Quicklook stats and thumbnails help triage which locations deserve deeper design work.

**Objective.** Generate site-level diagnostic metrics and snapshots.
    
**Inputs.**
* Project points (CSV/GeoPackage as per `config.py`).
  
**Outputs.**
* `tables/{AOI}_site_audit_points.csv` (+ optional PNGs in `outputs/figs/`).
  
**Assumptions.**
* Points in EPSG:4326; rasters sampled with nearest or bilinear.
  
**Knobs.**
* Sampling radius / list of rasters to sample (if exposed).


In [8]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_05_site_audit_points"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


21:57:48 | INFO | Site audit inputs loaded | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
21:57:48 | INFO | Loaded 39 site(s) from ago_poi_huambo_projectloc_dm_p.shp
21:57:48 | INFO | Neighborhood kernel: radius=5 cell(s) | cells_in_kernel=81
21:57:48 | INFO | Saved site audit → /mnt/c/Users/benny/OneDrive/Documents/Github/ago-lobitocorridor-analysis/outputs/tables/huambo_site_audit_points.csv | rows=39 | skipped_outside_grid=0
21:57:48 | INFO | Step 05 complete.


## Step 06 — Municipality (Admin2) ingest & correlations

Ingest Admin2 (RAPP) themes, standardize units, rasterize selected variables, and test correlations versus rural poverty. Produces a tidy “profiles” table and 1-km Admin2 surfaces usable in later steps.

**Objective.** Normalize Admin2 indicators and rasterize key variables.
    
**Inputs.**
* `data/vectors/admin2_rapp/ago_*_rapp_2020_a.shp` (themes in `THEME_VARS`), `PARAMS.TARGET_GRID`.
  
**Outputs.**
* `tables/{AOI}_municipality_indicators.csv` (wide),
* `tables/{AOI}_corr_with_rural_poverty.csv`,
* `tables/{AOI}_municipality_profiles.csv`,
* `rasters/{AOI}_muni_{theme}_{var}_1km.tif`.
  
**Assumptions & design.**
* Percentages normalized to 0–1 (`RAPP_PCT_IS_0_100=True`); travel time hours → minutes; `all_touched=True`.
* Missing variables skipped with warnings.
  
**Knobs.**
* `THEME_VARS` mapping, list of variables to rasterize.


In [22]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_06_muni_ingest"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
import config
importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
importlib.reload(m)

m.main()


06:24:23 | INFO | Target grid loaded | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
06:24:23 | INFO | Step 06 will skip themes: ('climevents',)
06:24:24 | INFO | Theme=waterresources: read=0.75s, rename=0.00s, normalize=0.00s, rows=128, cols=10
06:24:25 | INFO | Theme=communications: read=0.46s, rename=0.00s, normalize=0.00s, rows=128, cols=11
06:24:25 | INFO | Theme=infra: read=0.63s, rename=0.00s, normalize=0.01s, rows=128, cols=15
06:24:26 | INFO | Theme=foodinsecurity: read=0.65s, rename=0.00s, normalize=0.00s, rows=128, cols=14
06:24:27 | INFO | Theme=outflow: read=0.72s, rename=0.00s, normalize=0.00s, rows=128, cols=12
06:24:27 | INFO | Theme=poverty: read=0.72s, rename=0.00s, normalize=0.00s, rows=128, cols=8
06:24:28 | INFO | Theme=productions: read=0.71s, rename=0.00s, normalize=0.01s, rows=128, cols=15
06:24:29 | INFO | Theme=traveltime: read=0.70s, rename=0.00s, normalize=0.00s, rows=128, cols=6
06:24:29 | INFO | Ingested admin2 themes: ['communications', 'foodinsecurit

KeyError: "['geometry'] not found in axis"

## Step 07 — Priority surface (tunable, authoritative)

Create the **production** priority surface with coherent normalization, optional equity/poverty overlays, focal smoothing, tiny-cluster removal, and a Top-X% mask.

**Objective.** Build the authoritative priority map and mask with flexible weights.

**Inputs.**
* 1-km base layers + optional Admin2 rasters (auto-discovered), including `rwi_meta_1km` if present.
  
**Outputs.**
* `rasters/{AOI}_priority_score_0_1.tif`, `rasters/{AOI}_priority_top10_mask.tif`.
  
**Assumptions & design.**
* Unified normalization, masks, focal smoothing, small-cluster removal, Top-X selection.
  
**Knobs.**
* Weights and Top-X% in `PARAMS`/module (e.g., `PARAMS.TOP_X_PCT`).
  
**Note.**
* Downstream steps (11, 12) expect **these** canonical filenames from `config.py`.


In [17]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_07_priority_tunable"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config, utils_geo
# importlib.reload(config)
# importlib.reload(utils_geo)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
importlib.reload(m)

m.main()


09:13:43 | INFO | Target grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
09:13:43 | INFO | Reprojected raster to match grid: POP
09:13:43 | INFO | Reprojected raster to match grid: VEG
09:13:43 | INFO | Reprojected raster to match grid: NTL
09:13:43 | INFO | Reprojected raster to match grid: DRT
09:13:43 | INFO | Reprojected raster to match grid: CROP
09:13:43 | INFO | Reprojected raster to match grid: RURAL
09:13:43 | INFO | Reprojected overlay to match grid: huambo_rwi_meta_1km.tif
09:13:43 | INFO | Priority weight blend → ACC:0.41, POP:0.29, DRT:0.12, RWI:0.18
09:13:44 | INFO | Wrote huambo_priority_score_0_1.tif
09:13:44 | INFO | Wrote huambo_priority_top10_mask.tif | selected=111 cells
09:13:44 | INFO | Wrote sidecar meta → huambo_priority_score_0_1.meta.json
09:13:44 | INFO | Step 07 complete.


## Step 08 — Project-level KPIs

Aggregate access/exposure/equity metrics over project buffers or polygons. Useful for comparing alternatives under the same data stack.

**Objective.** Compute KPI summaries for each project footprint/buffer.
    
**Inputs.**
* Project geometries + aligned rasters (POP/CROP/ELECTRIC/PRIORITY/etc.).
  
**Outputs.**
* `tables/{AOI}_project_kpis.csv`.
  
**Assumptions.**
* Buffers in meters; stats computed with consistent nodata handling.
  
**Knobs.**
* Buffer distance; KPI list (in step module).


In [18]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_08_project_kpis"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config, utils_geo
# importlib.reload(config)
# importlib.reload(utils_geo)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


09:13:48 | INFO | Template grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
09:13:49 | INFO | Reprojected pop to match grid
09:13:49 | INFO | Reprojected cropf to match grid
09:13:49 | INFO | Reprojected grid to match grid
09:13:49 | INFO | Reprojected rural to match grid
09:13:49 | INFO | Reprojected prio to match grid
09:13:49 | INFO | Reprojected risk to match grid
09:13:49 | INFO | Reprojected pov to match grid
09:13:49 | INFO | Denominators | pop_total=2,997,196 | cropland_total_km2=8527.64 | electrified_cells_total=588 | cell_area_km2=0.840
09:13:49 | INFO | Loaded 39 project(s) from ago_poi_huambo_projectloc_dm_p.shp
09:13:49 | INFO | Saved project KPIs → huambo_project_kpis.csv | rows=39
09:13:49 | INFO | Step 08 complete.


## Step 09 — Municipality (Admin2) targeting

Rank Admin2 units using access (≤60/120 min), poverty/equity signals, and priority shares. Produces an actionable shortlist for piloting.

**Objective.** Score and rank Admin2s for investment targeting.

**Inputs.**
* Admin2 rasters and tabular indicators from Step 06 + travel-time surface.
  
**Outputs.**
* `tables/{AOI}_muni_targeting.csv` (+ optional choropleth figs).
  
**Assumptions.**
* Uses `%≤60/≤120m` coverage, priority mask share, etc.
  
**Knobs.**
* Thresholds in `PARAMS.ISO_THRESH`; ranking weights in the module.


In [19]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_09_muni_targeting"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


09:13:53 | INFO | Template grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
09:13:53 | INFO | Using admin2 geometry from ago_pop_huambo_adm2_poverty_rapp_2020_a.shp
09:14:29 | INFO | Collapsed Step 06 table from 904604 to 39 rows (unique ADM2).
09:14:29 | INFO | Saved municipality targeting table → huambo_priority_muni_rank.csv | rows=11
09:14:29 | INFO | Step 09 complete.


## Step 10 — Priority scenarios

Stress-test the priority model with alternative weight sets (e.g., stronger equity vs. market access) and compare outcomes.

**Objective.** Generate and record scenario variants of the priority surface.
    
**Inputs.**
* Same base overlays as Step 07, different weight/toggle sets.
  
**Outputs.**
* `rasters/{AOI}_priority_score_scnX.tif`, masks, and a `tables/{AOI}_priority_scenarios.csv`.
  
**Assumptions.**
* Scenarios defined in code or `PARAMS`.
  
**Knobs.**
* Scenario configs (weights/toggles) in the module or `PARAMS`.


In [20]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_10_priority_scenarios"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
importlib.reload(m)

m.main()


09:14:29 | INFO | Target grid | CRS=EPSG:4326 | size=280x216 | cell=0.0083x0.0083
09:14:30 | INFO | Using on-disk baseline: huambo_priority_top10_mask.tif
09:14:30 | INFO | Scenario [baseline] — ACC+POP+DRT, rural-only, min crop 5%, 3x3 smooth, Top10%
09:14:30 | INFO | Scenario [no_drought] — ACC+POP only; rural-only; min crop 5%; Top10%
09:14:30 | INFO | Scenario [veg_signal] — ACC+POP+VEG; rural-only; min crop 10%; Top10%
09:14:30 | INFO | Scenario [top_800km2] — ACC+POP+DRT, rural-only; select fixed 800 km²; mild smooth
09:14:30 | INFO | Wrote scenario summary → huambo_priority_scenarios_summary.csv | scenarios=4 | masks_saved=4
09:14:30 | INFO | Wrote scenarios sidecar → huambo_priority_scenarios.meta.json
09:14:30 | INFO | Step 10 complete.


## Step 11 — Priority clusters

Convert the Top-X% priority mask into connected clusters and compute cluster KPIs (population, cropland, RWI, and share within ≤30/60/120 min). Small, noisy blobs are removed using cell and km² thresholds.

**Objective.** Extract cluster polygons/labels and summarize them for shortlisting.
    
**Inputs.**
* `PRIORITY_TOP10_TIF` from `config.py`, `traveltime_min` + base rasters, optional `rwi_meta_1km`.
  
**Outputs.**
* `rasters/{AOI}_priority_clusters_1km.tif` (via `PRIORITY_CLUSTERS_TIF`),
* `tables/{AOI}_priority_clusters.csv`.
  
**Assumptions & design.**
* Prune tiny clusters by min cells and **km²** (now configurable via `PARAMS.MIN_CLUSTER_KM2`).
* Added KPIs: `share_le_30m/60m/120m` (uses `PARAMS.ISO_THRESH`).
  
**Knobs.**
* `PARAMS.MIN_CLUSTER_CELLS`, `PARAMS.MIN_CLUSTER_KM2`, `PARAMS.ISO_THRESH`.


In [ ]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_11_priority_clusters"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


## Step 12 — Travel-time catchments

Build catchment masks for each threshold and compute POP/CROP/RWI KPIs plus mean travel minutes—a quick validation of how “tight” the catchments are.

**Objective.** Derive per-threshold catchment KPIs and diagnostics.
    
**Inputs.**
* `{AOI}_traveltime_min.tif`, 1-km POP/CROPLAND, optional `rwi_meta_1km`.
  
**Outputs.**
* `tables/{AOI}_catchments_kpis.csv` (via `CATCHMENTS_KPI_CSV`),
* per-threshold masks in `rasters/` (if the step writes them).
  
**Assumptions & design.**
* Coverage = `tt <= thr`; includes `mean_travel_min` diagnostic.
  
**Knobs.**
* `PARAMS.ISO_THRESH`.


In [ ]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_12_traveltime_catchments"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


## Step 13 — Synergies overlay

Intersect priority with constraints/opportunities (e.g., flood risk, grid proximity, conservation). Useful to surface “no-regret” or “high-risk” zones.

**Objective.** Produce an overlay table (and optional rasters) capturing key synergies/tradeoffs.

**Inputs.**
* Priority rasters, flood/constraint rasters, grid/market proximity if available.
  
**Outputs.**
* `tables/{AOI}_synergies_overlay.csv`, optionally composited rasters.
  
**Assumptions.**
* Binary or categorical overlay logic documented in the step.
  
**Knobs.**
* Overlay weights/toggles in the module.


In [ ]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_13_synergies_overlay"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()


## Step 14 — OD-Lite (Admin2 gravity)

Sketch flows between Admin2 zones using a gravity model for corridor storytelling. Supports equity tilting (RWI), selectable impedance (exponential or power), distance cutoffs, and (optionally) doubly-constrained balancing.

**Objective.** Generate OD flows, zone attributes, and sampled agents for quick viz.
                                                                      
**Inputs.**
* `{AOI}_pop_1km.tif`, optional `rwi_meta_1km.tif`, Admin2 shapefile (`muni_path_for`).
  
**Outputs.**
* `tables/{AOI}_od_gravity.csv` (long: oi,dj,flow,dist_km),
* `tables/{AOI}_od_zone_attrs.csv` (zone masses + centroids),
* `tables/{AOI}_od_agents.csv` (sampled OD pairs for quick viz).
  
**Assumptions & design.**
* Geodesic centroid–centroid distances; **optional** RWI mass tilt;
* Impedance `f(D)` selectable (`exp` or `pow`); optional **doubly-constrained** via IPF; distance cutoff.
  
**Knobs (in `PARAMS`).**
* `OD_F`: `"exp"` or `"pow"`; `OD_LAMBDA` (exp), `OD_BETA` (pow),
* `OD_MAX_DIST_KM`, `OD_TRIPS_TOTAL`, `OD_USE_DOUBLY_CONSTRAINED`,
* `USE_RWI_IN_MASS`, `RWI_WEIGHT`, `OD_N_AGENTS`.


In [ ]:
import importlib

# >>> Change this to the step you want to run:
step_name = "step_14_lite_od"

# --- If you edited config.py (AOI, PATHS, PARAMS, etc.), UNCOMMENT BOTH lines below
# --- AND ALSO UNCOMMENT the importlib.reload(m) line further down.
# import config
# importlib.reload(config)

m = importlib.import_module(step_name)

# --- If you edited ONLY this step’s .py file, UNCOMMENT this line.
# --- If you edited config.py, ALSO UNCOMMENT this line after the two config lines above.
# importlib.reload(m)

m.main()
